In [1]:
from tkinter import *
import socket
import os
from _thread import *
import mysql.connector

In [2]:
def updateAccountMap():
    mydb = mysql.connector.connect(
        host='remotemysql.com',
        user="CoJ5g2XnKq",
        password="L6QhRYcFVP",
        database="CoJ5g2XnKq"
    )
    SQL = mydb.cursor()
    SQL.execute("select * from TaiKhoan")
    data=SQL.fetchall()
    global AccountMap
    AccountMap={x:y for x,y in data}

In [3]:
def registerAccount(ACT,PSW):
    mydb = mysql.connector.connect(
        host='remotemysql.com',
        user="CoJ5g2XnKq",
        password="L6QhRYcFVP",
        database="CoJ5g2XnKq"
    )
    SQL = mydb.cursor()
    SQL.execute(f"insert into TaiKhoan values ('{ACT}','{PSW}')")
    mydb.commit()

In [4]:
def ALL():
    mydb = mysql.connector.connect(
        host='remotemysql.com',
        user="CoJ5g2XnKq",
        password="L6QhRYcFVP",
        database="CoJ5g2XnKq"
    )
    SQL = mydb.cursor()
    SQL.execute("select * from Sach")
    data=SQL.fetchall()
    global QUERY
    QUERY=[[x,y,z,t] for x,y,z,t in data]

In [5]:
def QR(a,b):
    mydb = mysql.connector.connect(
        host='remotemysql.com',
        user="CoJ5g2XnKq",
        password="L6QhRYcFVP",
        database="CoJ5g2XnKq"
    )
    SQL = mydb.cursor()
    SQL.execute(f"select * from Sach where {a} like '%{b}%'")
    data=SQL.fetchall()
    global QUERY
    QUERY=[[x,y,z,t] for x,y,z,t in data]

In [6]:
class ServerForm:
        def __init__(self, master):
            master.geometry("700x700")
            listbox=Listbox(root,width=40, height=40)
            listbox.pack(pady=15)

In [7]:
root = Tk()
my_gui = ServerForm(root)
#root.mainloop()

In [ ]:
ServerSideSocket = socket.socket()
host = ''
port = 2004
ClientCount = 0
AccountMap={}
QUERY={}
updateAccountMap()
try:
    ServerSideSocket.bind((host, port))
except socket.error as e:
    print(str(e))

print('The server is on.....')
ServerSideSocket.listen(5)

def multi_threaded_sever(connection,stt):
    connection.sendall(str.encode('ACCEPTED CONNECT'))
    while True:
        try:
            data = connection.recv(2048)
        except ConnectionResetError:
            print(f"CLIENT {stt} DISCONNECTED: (ConnectionResetError)")
            break
        if not data:
            break
        data=data.decode('utf-8')
        print(f'Server received from client {stt}: {data}')
        response = ""
        if data=='NEW CLIENT CONNECTED':
            response = f'HELLO CLIENT {stt}'
        data=data.split()
        if data[0]=='QUERY':
            if data[1]=='ALL':
                ALL()
            if data[1]=='SELECT':
                pass
            if data[1][0]=='F':
                QR(data[1],data[2]);
            response=""
            for i in QUERY:
                for j in i:
                    response+=j+','
                response+='\n'
                pass
        if data[0]=='REQUEST':
            if data[1]=='REGISTER':
                if data[2] in AccountMap.keys():
                    response="DENIED REGISTER EXITED"
                else:
                    response="ACCEPTED REGISTER"
                    registerAccount(data[2],data[3])
                updateAccountMap()
            if data[1]=='LOGIN':
                while len(data)<4:
                    data.append("")
                if data[2] in AccountMap.keys() and data[3] == AccountMap[data[2]]:
                    response = "ACCEPTED LOGIN"
                else:
                    response = "DENIED LOGIN"
        connection.sendall(str.encode(response))
        print(f'Server sended to client {stt}: ' + response)
        
    global ClientCount
    ClientCount=ClientCount-1
    connection.close()
    print(f"CLIENT {stt} DISCONNECTED")

stt=0
while True:
    Client, address = ServerSideSocket.accept()
    print('Connected to: ' + address[0] + ':' + str(address[1]))
    start_new_thread(multi_threaded_sever, (Client,stt,))
    stt+=1
    ClientCount += 1
    print('Thread Number: ' + str(ClientCount))
ServerSideSocket.close()

The server is on.....
Connected to: 127.0.0.1:39664
Thread Number: 1
Server received from client 0: NEW CLIENT CONNECTED
Server sended to client 0: HELLO CLIENT 0
Server received from client 0: REQUEST LOGIN 123 1
Server sended to client 0: ACCEPTED LOGIN
Server received from client 0: QUERY ALL
Server sended to client 0: 001,Kinh te 1,Le Minh Tu,Kinh te,
002,Kinh te 2,Nguyen Xuan Vy,Kinh te,
003,Toan 1,Nguyen Xuan Vy,Toan,
004,Toan cao cap,Nguyen Xuan Vy,Toan,

Server received from client 0: QUERY F_ID 002
Server sended to client 0: 002,Kinh te 2,Nguyen Xuan Vy,Kinh te,

Server received from client 0: QUERY F_NAME Kinh te
Server sended to client 0: 001,Kinh te 1,Le Minh Tu,Kinh te,
002,Kinh te 2,Nguyen Xuan Vy,Kinh te,

Server received from client 0: QUERY F_AUTHOR Le Minh Tu
Server sended to client 0: 001,Kinh te 1,Le Minh Tu,Kinh te,

Server received from client 0: QUERY F_AUTHOR Nguyen Xuan Vy
Server sended to client 0: 002,Kinh te 2,Nguyen Xuan Vy,Kinh te,
003,Toan 1,Nguyen Xuan V